In [ ]:
!pip3 install einops

In [ ]:
import torch
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


import math
import h5py
import numpy as np
from PIL import Image
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import imageio, warnings, os

# from interpretation_methods import *
# from utils.imagenet_seg_loader import ImagenetSegLoader
# from utils.model_loaders import vit_base_patch16_224_dino, vit_base_patch16_224
# from utils.input_arguments import get_arg_parser
# from utils.saver import Saver
# from utils.sideplot import side_plot
# from utils.image_denorm import image_vizformat

warnings.filterwarnings("ignore")

# <center><font size="+6"> **1) Building the model**</font>

## **Config Setup**

In [2]:
def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


model_base_dir = "/kaggle/input/popular-vits-for-interpretation/pytorch/default/1"
default_config = {
    # patch models
    'vit_small_patch16_224': _cfg(
        url=f'{model_base_dir}/vit_small_p16_224-15ec54c9.pth',
    ),
    'vit_base_patch16_224': _cfg(
        url=f'{model_base_dir}/jx_vit_base_p16_224-80ecf9dd.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_base_patch16_224_moco': _cfg(
        url=f'{model_base_dir}/linear-vit-b-300ep.pth.tar',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_base_patch16_224_dino': _cfg(
        url_backbone=f'{model_base_dir}/dino_vitbase16_pretrain.pth',
        url_linear=f'{model_base_dir}/dino_vitbase16_linearweights.pth',
        url=f'{model_base_dir}/dino_vitbase16_pretrain_full_checkpoint.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_mae_patch16_224': _cfg(
        url=f'{model_base_dir}/mae_finetuned_vit_base.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5),
    ),
    'vit_large_patch16_224': _cfg(
        url=f'{model_base_dir}/jx_vit_large_p16_224-4ee7a4dc.pth',
        mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
}

cls = ['airplane', 'bicycle', 'bird',
       'boat',
       'bottle',
       'bus',
       'car',
       'cat',
       'chair',
       'cow',
       'dining table',
       'dog',
       'horse',
       'motobike',
       'person',
       'potted plant',
       'sheep',
       'sofa',
       'train',
       'tv'
       ]

# **Image Loader**

In [ ]:
class ImagenetSegLoader(data.Dataset):
    CLASSES = 2

    def __init__(self, path, length_limit=None, transform=None, target_transform=None):
        self.path = path
        self.transform = transform
        self.target_transform = target_transform
        self.h5py = None
        tmp = h5py.File(path, 'r')

        if length_limit:
            self.data_length = length_limit
        else:
            self.data_length = len(tmp['/value/img'])

        tmp.close()
        del tmp

    def __getitem__(self, index):

        if self.h5py is None:
            self.h5py = h5py.File(self.path, 'r')

        img = np.array(self.h5py[self.h5py['/value/img'][index, 0]]).transpose((2, 1, 0))
        target = np.array(self.h5py[self.h5py[self.h5py['/value/gt'][index, 0]][0, 0]]).transpose((1, 0))

        img = Image.fromarray(img).convert('RGB')
        target = Image.fromarray(target)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = np.array(self.target_transform(target)).astype('int32')
            target = torch.from_numpy(target).long()

        return img, target

    def __len__(self):
        return self.data_length


def image_vizformat(image):
    inv_normalize = transforms.Normalize(
        mean=[-0.5 / 0.5, -0.5 / 0.5, -0.5 / 0.5],
        std=[1 / 0.5, 1 / 0.5, 1 / 0.5]
    )

    img = inv_normalize(image[0])
    img = torch.permute(img, (1, 2, 0))
    img = img.detach().cpu().numpy()
    return img


## **Pretrained Models loader function**

In [7]:
import logging
import os
import math
from collections import OrderedDict
from copy import deepcopy
from typing import Callable

import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

_logger = logging.getLogger(__name__)

"""
     Load pretrained model.
     
     For dino, the url_linear contain weight an bias for the head that needed to be injected into backbone. 
     Cause, the backbone doesn't have a classifier. Instead, is a very good feature extractor.
     
     No other model type require this processing.
     
"""


def load_pretrained(model, cfg=None, num_classes=1000, in_chans=3, filter_fn=None, strict=True, mae=False, moco=False, dino=False):
    if cfg is None:
        cfg = getattr(model, 'default_cfg')

    if dino:
        state_backbone = torch.load(cfg['url_backbone'], map_location='cpu')
        state_linear = torch.load(cfg['url_linear'], map_location='cpu')[
            'state_dict']  # Get weight of the last layer only.
        state_dict = state_backbone.copy()
        state_dict['head.weight'] = state_linear[
            'module.linear.weight']  # Shape: (1000, 1536). That means, the final layer would have input from previous fc layer with 1536 neurons.
        state_dict['head.bias'] = state_linear['module.linear.bias']  # Shape: (1000,)
    else:
        state_dict = torch.load(cfg['url'], map_location='cpu')

    if mae:
        state_dict = state_dict['model']
    if moco:
        state_dict = state_dict['state_dict']
        for i in list(state_dict.keys()):
            name = i.split('module.')[1]
            state_dict[name] = state_dict.pop(i)

    if filter_fn is not None:
        state_dict = filter_fn(state_dict)

    if in_chans == 1:
        conv1_name = cfg['first_conv']
        _logger.info('Converting first conv (%s) pretrained weights from 3 to 1 channel' % conv1_name)
        conv1_weight = state_dict[conv1_name + '.weight']
        # Some weights are in torch.half, ensure its float for sum on CPU
        conv1_type = conv1_weight.dtype
        conv1_weight = conv1_weight.float()
        O, I, J, K = conv1_weight.shape
        if I > 3:
            assert conv1_weight.shape[1] % 3 == 0
            conv1_weight = conv1_weight.reshape(O, I // 3, 3, J, K)  # For models with space2depth stems
            conv1_weight = conv1_weight.sum(dim=2, keepdim=False)
        else:
            conv1_weight = conv1_weight.sum(dim=1, keepdim=True)
        conv1_weight = conv1_weight.to(conv1_type)
        state_dict[conv1_name + '.weight'] = conv1_weight
    elif in_chans != 3:
        conv1_name = cfg['first_conv']
        conv1_weight = state_dict[conv1_name + '.weight']
        conv1_type = conv1_weight.dtype
        conv1_weight = conv1_weight.float()
        O, I, J, K = conv1_weight.shape
        if I != 3:
            _logger.warning('Deleting first conv (%s) from pretrained weights.' % conv1_name)
            del state_dict[conv1_name + '.weight']
            strict = False
        else:
            # NOTE this strategy should be better than random init, but there could be other combinations of
            # the original RGB input layer weights that'd work better for specific cases.
            _logger.info('Repeating first conv (%s) weights in channel dim.' % conv1_name)
            repeat = int(math.ceil(in_chans / 3))
            conv1_weight = conv1_weight.repeat(1, repeat, 1, 1)[:, :in_chans, :, :]
            conv1_weight *= (3 / float(in_chans))
            conv1_weight = conv1_weight.to(conv1_type)
            state_dict[conv1_name + '.weight'] = conv1_weight

    classifier_name = cfg['classifier']
    if num_classes == 1000 and cfg['num_classes'] == 1001:  # special case for imagenet trained models with extra background class in pretrained weights
        classifier_weight = state_dict[classifier_name + '.weight']
        state_dict[classifier_name + '.weight'] = classifier_weight[
                                                  1:]  # Class index 0 give to background. Therefore, starting from index 1.
        classifier_bias = state_dict[classifier_name + '.bias']
        state_dict[classifier_name + '.bias'] = classifier_bias[1:]
    elif num_classes != cfg['num_classes']:
        # completely discard fully connected for all other differences between pretrained and created model
        del state_dict[classifier_name + '.weight']
        del state_dict[classifier_name + '.bias']
        strict = False

    model.load_state_dict(state_dict, strict=strict)


"""
    Load state dictionary of the pretrained model. 

"""


def load_state_dict(checkpoint_path, use_ema=False):
    if checkpoint_path and os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        state_dict_key = 'state_dict'
        if isinstance(checkpoint, dict):
            if use_ema and 'state_dict_ema' in checkpoint:
                state_dict_key = 'state_dict_ema'
        if state_dict_key and state_dict_key in checkpoint:
            new_state_dict = OrderedDict()
            for k, v in checkpoint[state_dict_key].items():
                # strip `module.` prefix
                name = k[7:] if k.startswith('module') else k
                new_state_dict[name] = v
            state_dict = new_state_dict
        else:
            state_dict = checkpoint
        _logger.info("Loaded {} from checkpoint '{}'".format(state_dict_key, checkpoint_path))
        return state_dict
    else:
        _logger.error("No checkpoint found at '{}'".format(checkpoint_path))
        raise FileNotFoundError()

        
  
"""

    @@@@ From model_loaders.py

"""

def _conv_filter(state_dict, patch_size=16):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v
    return out_dict


def vit_small_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=8, num_heads=6, mlp_ratio=3, qkv_bias=False,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=8, num_heads=6, mlp_ratio=3, qkv_bias=False,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)

    model.default_cfg = default_config['vit_small_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter)
    return model


def vit_base_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_config['vit_base_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter)
    return model


def vit_base_patch16_224_dino(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      dino=True, **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), dino=True, **kwargs)

    model.default_cfg = default_config['vit_base_patch16_224_dino']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter, dino=True)
    return model


def vit_base_patch16_224_moco(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)

    model.default_cfg = default_config['vit_base_patch16_224_moco']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter, moco=True)
    return model


def vit_mae_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                      mae=True, **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), global_pooling=True, **kwargs)
    model.default_cfg = default_config['vit_mae_patch16_224']
    if pretrained:
        load_pretrained(
            model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3), filter_fn=_conv_filter, mae=True)
    return model


def vit_large_patch16_224(pretrained=False, w_rel=False, **kwargs):
    if w_rel:
        model = VisionTransformerWRel(patch_size=16, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4, qkv_bias=True,
                                      **kwargs)
    else:
        model = VisionTransformer(patch_size=16, embed_dim=1024, depth=24, num_heads=16, mlp_ratio=4, qkv_bias=True,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-6), **kwargs)
    model.default_cfg = default_config['vit_large_patch16_224']
    if pretrained:
        load_pretrained(model, num_classes=model.num_classes, in_chans=kwargs.get('in_chans', 3))
    return model


## **Input arguments - input_arguments.py**

In [8]:
import argparse


def get_arg_parser():
    parser = argparse.ArgumentParser(description='Training multi-class classifier')
    parser.add_argument('--arc', type=str, default='vgg', metavar='N',
                        help='Model architecture')
    parser.add_argument('--train_dataset', type=str, default='imagenet', metavar='N',
                        help='Testing Dataset')
    parser.add_argument('--method', type=str,
                        default='ours_c',
                        choices=['rollout', 'lrp', 'partial_lrp', 'transformer_attribution', 'attn_last_layer',
                                 'attn_gradcam', 'generic_attribution', 'ours', 'ours_c'],
                        help='')
    parser.add_argument('--thr', type=float, default=0.,
                        help='threshold')
    parser.add_argument('--start_layer', type=int, default=4,
                        help='start_layer')
    parser.add_argument('--K', type=int, default=1,
                        help='new - top K results')
    parser.add_argument('--save-img', action='store_true',
                        default=True,
                        help='')
    parser.add_argument('--no-ia', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-fx', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-fgx', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-m', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--no-reg', action='store_true',
                        default=False,
                        help='')
    parser.add_argument('--is-ablation', type=bool,
                        default=False,
                        help='')
    parser.add_argument('--len-lim', type=int,
                        default=1,
                        help='')
    parser.add_argument('--imagenet-seg-path', type=str, required=True)

    return parser


# <font color="#DC143C"> **Building the Vision Transformer model**

## **Initializing the weight**

In [10]:
def _no_grad_trunc_normal_(tensor, mean, std, a, b):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated by using a truncated uniform distribution and
        # then using the inverse CDF for the normal distribution.
        # Get upper and lower cdf values
        l = norm_cdf((a - mean) / std)
        u = norm_cdf((b - mean) / std)

        # Uniformly fill tensor with values from [l, u], then translate to
        # [2l-1, 2u-1].
        tensor.uniform_(2 * l - 1, 2 * u - 1)

        # Use inverse cdf transform for normal distribution to get truncated
        # standard normal
        tensor.erfinv_()

        # Transform to proper mean, std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)

        # Clamp to ensure it's in the proper range
        tensor.clamp_(min=a, max=b)
        return tensor


def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # type: (Tensor, float, float, float, float) -> Tensor
    r"""Fills the input Tensor with values drawn from a truncated
    normal distribution. The values are effectively drawn from the
    normal distribution :math:`\mathcal{N}(\text{mean}, \text{std}^2)`
    with values outside :math:`[a, b]` redrawn until they are within
    the bounds. The method used for generating the random values works
    best when :math:`a \leq \text{mean} \leq b`.
    Args:
        tensor: an n-dimensional `torch.Tensor`
        mean: the mean of the normal distribution
        std: the standard deviation of the normal distribution
        a: the minimum cutoff value
        b: the maximum cutoff value
    Examples:
        >>> w = torch.empty(3, 5)
        >>> nn.init.trunc_normal_(w)
    """
    return _no_grad_trunc_normal_(tensor, mean, std, a, b)

## **MLP**

In [ ]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

## **Attention**